In [1]:
import mip
import sys
class Pavage:
    def __init__(self,x,y) -> None:
        self.x=x
        self.y=y
        self.cases=[]
        
        self.tripletList=[]
        #Filling Cases
        for i in range(x):
            for j in  range(y):
                self.cases.append((i,j))
        self.casesWeight=[0 for i in self.cases]
        #Filling tripletSet
        for a in range(len(self.cases)):
            for b in range(len(self.cases)):
                for c in range(len(self.cases)):
                    if(a!=b and b!=c):
                        stepToOthersCasesB=abs(self.cases[a][0]-self.cases[b][0])+abs(self.cases[a][1]-self.cases[b][1])
                        stepToOtherCasesC=abs(self.cases[a][0]-self.cases[c][0])+abs(self.cases[a][1]-self.cases[c][1])
                        if(stepToOthersCasesB==1 and stepToOtherCasesC==1):#a b c sont voisins
                            triplet=[list(self.cases[a]),list(self.cases[b]),list(self.cases[c])]
                            triplet.sort()
                            self.tripletList.append((tuple(triplet[0]),tuple(triplet[1]),tuple(triplet[2])))
        self.tripletSet=set(self.tripletList)
        self.tripletList=list(self.tripletSet)
        self.tripletVariable=[0 for i in self.tripletList]

pavage=Pavage(2,2)
print(pavage.tripletList)
print(len(pavage.tripletList))



[((0, 1), (1, 0), (1, 1)), ((0, 0), (1, 0), (1, 1)), ((0, 0), (0, 1), (1, 0)), ((0, 0), (0, 1), (1, 1))]
4


In [4]:
def solve(pavage):
    m=mip.Model()
    n=pavage.x
    p=pavage.y
    contraintes=[]#Initialisation du talbeau de contraintes
    for i in range(n):
        for j in  range(p):
           contraintes.append([])
    #Création des variables
    #Variables sur toutes les tuiles
    tuilesWeight=[]
    for triplet in pavage.tripletList:
        tuilesWeight.append(len(triplet))
    varX=[]
    indice=0
    for x in pavage.tripletList:
        varIntermediaire=m.add_var(var_type=mip.BINARY, name=("varX"+str(indice)))
        varX.append(varIntermediaire)
        for triplet in x:
            contraintes[triplet[1]][triplet[0]].append(varIntermediaire)
        indice+=1
    for i in range(len(contraintes)):
        for j in range(len(contraintes[i])):
            m += mip.xsum(contraintes[i][j]) <= 1
    m.objective = mip.maximize(mip.xsum(varX[i]*tuilesWeight[i] for i in range(len(varX))))
    maximal=m.optimize(max_seconds=60)
    print(maximal)
    return m